In [5]:
import sys 
import os 
import pandas as pd


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.getLatestFiles import get_latest_master_players, get_latest_master_matches, get_latest_master_events


In [6]:
master_players_df = get_latest_master_players()
master_matches_df = get_latest_master_matches()
master_events_df = get_latest_master_events()



✅ 2697 players found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Players/20251119_master_players.csv
✅ 23882 matches found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Matches/20251120_master_matches.csv 
✅ 188 events found in latest MASTER: /home/mdannegarn/table_tennis_stats/Data/Master/Events/20251119_master_events.csv 


In [15]:
tables_matches_df = pd.merge(master_matches_df, master_events_df[["TableSponsor", "eventId"]], how='left', on='eventId')

In [26]:
df =tables_matches_df


In [20]:
tables_matches_df.dropna(subset=['TableSponsor'])

,eventId,documentCode,subEventName,subEventDescription,venueName,tableName,dnf,ttrReview,serverNext,duration (unreliable),...,loser1Wins,loser4Wins,winnerAvgWinningMargin,loserAvgWinningMargin,comebackBy2,comebackBy3,winnerAvgPointsPerSet,loserAvgPointsPerSet,Round,TableSponsor
0,2410,TTEMSINGLES-----------RND1000700--,Men Singles,Men's Singles - Preliminary Round 1 - Match 7,Lusail Sports Arena,Table 6,False,NaN,NaN,00:15:15,...,0.0,0.0,2.33,0.0,False,False,12.00,9.67,prelim,Double Fish
1,2410,TTEMSINGLES-----------RND1002600--,Men Singles,Men's Singles - Preliminary Round 1 - Match 26,Lusail Sports Arena,Table 4,False,NaN,NaN,00:00:21,...,0.0,0.0,5.67,0.0,False,False,11.33,5.67,prelim,Double Fish
2,2410,TTEMSINGLES-----------RND1004600--,Men Singles,Men's Singles - Preliminary Round 1 - Match 46,Lusail Sports Arena,Table 3,False,NaN,NaN,00:00:00,...,0.0,0.0,2.33,0.0,False,False,11.00,8.67,prelim,Double Fish
3,2410,TTEMSINGLES-----------RND1001900--,Men Singles,Men's Singles - Preliminary Round 1 - Match 19,Lusail Sports Arena,Table 5,False,NaN,NaN,00:27:19,...,0.0,0.0,3.00,0.0,False,False,12.33,9.33,prelim,Double Fish
4,2410,TTEMSINGLES-----------RND1001000--,Men Singles,Men's Singles - Preliminary Round 1 - Match 10,Lusail Sports Arena,Table 7,False,NaN,NaN,00:13:28,...,0.0,0.0,3.67,0.0,False,False,12.00,8.33,prelim,Double Fish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23877,3176,TTEMSINGLES-----------R64-003000----------,Men's Singles,Men's Singles - Round of 64 - Match 30,Sultan Qaboos Sport Complex,Table 3,False,False,133251.0,00:18:44,...,0.0,0.0,6.67,0.0,False,False,11.00,4.33,R<built-in function round>,Butterfly
23878,3176,TTEWSINGLES-----------R64-002700----------,Women's Singles,Women's Singles - Round of 64 - Match 27,Sultan Qaboos Sport Complex,Table 2,False,False,132144.0,00:34:26,...,0.0,0.0,3.33,0.0,False,False,11.33,8.00,R<built-in function round>,Butterfly
23879,3176,TTEMSINGLES-----------R64-003100----------,Men's Singles,Men's Singles - Round of 64 - Match 31,Sultan Qaboos Sport Complex,Table 1,False,False,122002.0,00:29:33,...,0.0,0.0,3.67,3.0,False,False,11.00,9.00,R<built-in function round>,Butterfly
23880,3176,TTEWSINGLES-----------R64-002300----------,Women's Singles,Women's Singles - Round of 64 - Match 23,Sultan Qaboos Sport Complex,Table 4,False,False,135391.0,00:45:18,...,0.0,0.0,3.67,4.5,False,False,9.80,9.40,R<built-in function round>,Butterfly


In [25]:

style_map = master_players_df.set_index('playerId')['Style'].to_dict()
tables_matches_df['winner_style'] = tables_matches_df['winnerId'].map(style_map)
tables_matches_df['loser_style'] = tables_matches_df['loserId'].map(style_map)

In [27]:
df['attacker_won'] = df['winner_style'].str.contains('Attack', case=False, na=False)

In [ ]:
# --- Feature Engineering ---

# 1. Identify if the Winner was an Attacker
# (Assuming style strings are 'Attacker', 'Defender')
df['attacker_won'] = df['winner_style'].str.contains('Attack', case=False, na=False)

# 2. Calculate "Tightness" (Total Points per Set)
# You already calculated 'total_points' and 'total_sets' in your feature engineering
df['points_per_set'] = df['total_points'] / df['total_sets']

# --- Aggregation by Table Brand ---
table_stats = df.groupby('TableBrand').agg(
    matches_played=('eventId', 'count'),
    attacker_win_rate=('attacker_won', 'mean'),
    avg_points_per_set=('pointsPerSet', 'mean'),
    avg_winning_margin=('avgWinningMargin', 'mean') # From your Tier 3 stats
).reset_index()

# Filter for brands with a significant sample size (e.g., > 50 matches)
table_stats = table_stats[table_stats['matches_played'] > 50]

print(table_stats.head())

KeyError: 'total_points'